In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
import warnings
warnings.simplefilter('ignore')
from sklearn.preprocessing import LabelEncoder

Display example of data. Real data cannot be displayed due to confidentiality reasons. 

In [2]:
data = pd.read_csv('data.csv')

In [5]:
data.head()

,company_name_x,domain,category_list,Client_Focus__c,employee_count,age,Main_Client_Sector__c,Additional_Client_Sector__c,Business_Model__c,funding_rounds,funding_total_usd,investment_type,announced_on,raised_amount_usd,europe,eu,dach,nordics,western_europe,eastern_europe,southern_europe,north_america,asia,brics,baltics,cw_of_ind_states,lat_america,near_east,north_africa,ssh_africa,oceania
0,a,a.com,3d technology,b2b,0.0,8.117728,manufacturing,NaN,retail,1.0,NaN,seed,2014-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a,a.com,3d technology,b2b,0.0,8.117728,manufacturing,NaN,retail,1.0,NaN,seed,2014-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b,b.com,"bitcoin,e-commerce,fintech,gift card,mobile,so...",b2b,11.0,8.117728,trade,finance,services,8.0,12485633.0,angel,2013-02-11,1300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b,b.com,"bitcoin,e-commerce,fintech,gift card,mobile,so...",b2b,11.0,8.117728,trade,finance,services,8.0,12485633.0,series_unknown,2015-06-05,1549936.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b,b.com,"bitcoin,e-commerce,fintech,gift card,mobile,so...",b2b,11.0,8.117728,trade,finance,services,8.0,12485633.0,series_a,2013-12-13,1694279.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Shape of real data before preprocessing. A company might have multiple entries due to several funding rounds. Each row corresponds to one FR. All further computations made on real data.

In [4]:
data.shape

(694486, 31)

In [5]:
regions = [
        'europe', 'eu', 'dach', 'nordics','western_europe',
        'eastern_europe', 'southern_europe','north_america', 'asia', 'brics',
        'baltics', 'cw_of_ind_states', 'lat_america','near_east', 'north_africa',
        'ssh_africa', 'oceania'
    ]


def preprocess(data):
    # Create time passed since a funding round in years as diff between today and announce date
    data.announced_on = pd.to_datetime(data.announced_on)
    data['funding_ago'] = (pd.Timestamp.today() - data.announced_on).dt.days/365.25
    
    # Where age is missing add the time from the first funding round (if known)
    data['first_known_ago'] = data.groupby(['company_name_x', 'domain']).funding_ago.transform('max')
    data.age.fillna(data.first_known_ago, inplace=True)
    data.drop('first_known_ago', axis=1, inplace=True)
    
    
    # Encode investment types manually to preserve ordinal info
    invest_dict = {
        'angel':1, 'seed':2, 'series_a':3, 'series_b':4, 'series_c':4, 'series_d':4,
        'series_e':4, 'series_f':4, 'series_g':4, 'series_h':4, 'series_i':4
    }
    data.investment_type = data.investment_type.replace(invest_dict)
    # Encode all other investment types (unknown) as zero
    data.investment_type = pd.to_numeric(data.investment_type, errors='coerce').fillna(0)
    
    
    data.sort_values(['company_name_x', 'domain', 'announced_on'], ascending=False, inplace=True)

    
    # from the existing dataset leave only the last funding round (known or unknown) for each company
    data = data.drop_duplicates(['company_name_x', 'domain']).reset_index(drop=True)
    
    
    data.drop(['announced_on'], axis=1, inplace=True)
    
    
    # Deal with missing data - make a separate category 
    data.funding_ago.fillna(-1, inplace=True)
    data.age.fillna(-1, inplace=True)
    data.raised_amount_usd.fillna(-1, inplace=True)
    data.funding_total_usd.fillna(-1, inplace=True)
    data[regions] = data[regions].fillna(-1)
    
    # Leave only companies younger than 10 years
    data = data[(data.age < 10) & (data.age >= -1)]
    
    # Convert categorical data to numerical
    le = LabelEncoder()
    col_to_enc = ['Client_Focus__c', 'Main_Client_Sector__c', 'Business_Model__c']
    data[col_to_enc] = data[col_to_enc].fillna(0)
    data = data.apply(lambda x:
                      le.fit_transform(x.astype('str')) if x.name in col_to_enc else x
                     )   
    data.employee_count = le.fit_transform(data.employee_count.astype('int'))
    data = pd.concat([
        data, 
        data.Additional_Client_Sector__c.str.replace(' ', '').str.get_dummies(sep=';')
    ], axis=1)
    data.drop('Additional_Client_Sector__c', axis=1, inplace=True)
    data.drop('category_list', axis=1, inplace=True)
    return data

In [6]:
prep_data = preprocess(data)

In [7]:
prep_data.to_csv('prep_data.csv', index=False)

Data shape after preprocessing. Now each company has only one entry. Rows correspond to last FR of a company. 

In [8]:
prep_data.shape

(253506, 58)